In [ ]:
# https://habr.com/ru/post/677618/
# https://habr.com/ru/post/562064/
# https://colab.research.google.com/drive/1RgSJ0k-qppdtTEIR5_X1tp80ntfNXQh1?usp=sharing
# Тут про векторизацию текста

In [ ]:
# https://towardsdatascience.com/clustering-sentence-embeddings-to-identify-intents-in-short-text-48d22d3bf02e

# Почитать про кластеринг

### Тесты кода по кластеризации

In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install transformers sentencepiece datasets razdel -q

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
import razdel
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModel
from transformers.modeling_outputs import BaseModelOutput

In [ ]:
df = pd.read_excel('/kaggle/input/mts-ml-cup-sites-raw/sites.xlsx')
df_w_descr = df.loc[~df.text.isna()]

# df_w_descr = df_w_descr[0:500]

In [ ]:
df_w_descr

In [ ]:
replaces_re = [
['Быстрый ответ:.* К основным', 'К основным'],
['Добавлены результаты по запросу.*».', '']
]

replaces = [
'Преимущества. Потери трафика при переходе со смартфонов уменьшаются, даже если у пользователей медленный или нестабильный интернет.',
'Некоторые ссылки отсутствуют в результатах поиска в силу требований применимого законодательства.',
'Преимущества. Потери трафика при переходе со смартфонов уменьшаются, даже если у пользователей',
'Рост популярности Турбо-страниц у пользователей положительно влияет на посещаемость сайта.',
'К основным результатамОбратная связь о специальных возможностях',
'поиск картинки видео карты товары переводчик все сервисы',
'Волгоград Google Bing Сообщить об ошибке Настройки Я.ру',
'Владелец сайта предпочёл скрыть описание страницы.',
'Владелец сайта предпочёл описание страницы.',
'Обратная связь о специальных возможностях',
'Снижается нагрузка на хостинг и серверы',
'РКН: сайт нарушает закон РФ',
'К основным результатам',
'Навигационный ответ ',
'Навигационный ответ',
'Сообщить об ошибке',
'Статья в Википедии',
'все сервисы Меню ',
'Видимость сайта ',
'Исходящие ссылки',
'Смотрите также',
'Турбо-страница',
'Отменить Меню',
'Скрыть Меню ',
'Весь список',
'Скрыть Меню',
'Читать ещё',
'Люди ищут ',
'Подробнее',
'Отменить',
'Скрыть ',
'Меню',
]

for el in replaces_re:
    df_w_descr['text'] = df_w_descr['text'].str.replace(el[0], el[1])

for el in replaces:
    df_w_descr['text'] = df_w_descr['text'].str.replace(el, '')

In [ ]:
bert_name = 'cointegrated/LaBSE-en-ru'
enc_tokenizer = AutoTokenizer.from_pretrained(bert_name)
encoder = AutoModel.from_pretrained(bert_name)

if torch.cuda.is_available():
    encoder.cuda()

def encode(texts, do_norm=True):
    encoded_input = enc_tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        model_output = encoder(**encoded_input.to(encoder.device))
        embeddings = model_output.pooler_output
        if do_norm:
            embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings

In [ ]:
from tqdm._tqdm_notebook import tqdm_notebook

x = torch.empty(size=(df_w_descr.shape[0], 768))
for i in tqdm_notebook(range(df_w_descr.shape[0])):
    x[i] = encode(df_w_descr['text'][i:i+1].values[0])

y = x.cpu().detach().numpy()

In [ ]:
print(y.shape)
np.save('y.npy', y)

In [ ]:
!pip install hdbscan
!pip install umap-learn
!pip install cuml

In [ ]:
import random
from functools import partial

import matplotlib.pyplot as plt
# import hdbscan
import cuml.cluster as hdbscan
import umap
import tensorflow as tf
import tensorflow_hub as hub
# from sentence_transformers import SentenceTransformer
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score
from tqdm.notebook import trange
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval, Trials

##### Функции кластеризации

In [ ]:
def generate_clusters(message_embeddings,
                      n_neighbors,
                      n_components, 
                      min_cluster_size,
                      min_samples = None,
                      random_state = None):
    """
    Returns HDBSCAN objects after first performing dimensionality reduction using UMAP
    
    Arguments:
        message_embeddings: embeddings to use
        n_neighbors: int, UMAP hyperparameter n_neighbors
        n_components: int, UMAP hyperparameter n_components
        min_cluster_size: int, HDBSCAN hyperparameter min_cluster_size
        min_samples: int, HDBSCAN hyperparameter min_samples
        random_state: int, random seed
        
    Returns:
        clusters: HDBSCAN object of clusters
    """
    
    umap_embeddings = (umap.UMAP(n_neighbors = n_neighbors, 
                                n_components = n_components, 
                                metric = 'cosine', 
                                random_state=random_state)
                            .fit_transform(message_embeddings))

    clusters = hdbscan.HDBSCAN(min_cluster_size = min_cluster_size, 
                               min_samples = min_samples,
                               metric='euclidean', 
                               gen_min_span_tree=True,
                               cluster_selection_method='eom').fit(umap_embeddings)
    
    return clusters

In [ ]:
def score_clusters(clusters, prob_threshold = 0.05):
    """
    Returns the label count and cost of a given clustering

    Arguments:
        clusters: HDBSCAN clustering object
        prob_threshold: float, probability threshold to use for deciding
                        what cluster labels are considered low confidence

    Returns:
        label_count: int, number of unique cluster labels, including noise
        cost: float, fraction of data points whose cluster assignment has
              a probability below cutoff threshold
    """
    
    cluster_labels = clusters.labels_
    label_count = len(np.unique(cluster_labels))
    total_num = len(clusters.labels_)
    cost = (np.count_nonzero(clusters.probabilities_ < prob_threshold)/total_num)
    
    return label_count, cost

In [ ]:
def random_search(embeddings, space, num_evals):
    """
    Randomly search parameter space of clustering pipeline

    Arguments:
        embeddings: embeddings to use
        space: dict, contains keys for 'n_neighbors', 'n_components',
               and 'min_cluster_size' and values with
               corresponding lists or ranges of parameters to search
        num_evals: int, number of random parameter combinations to try

    Returns:
        df_result: pandas dataframe containing info on each evaluation
                   performed, including run_id, parameters used, label
                   count, and cost
    """
    
    results = []
    
    for i in trange(num_evals):
        n_neighbors = random.choice(space['n_neighbors'])
        n_components = random.choice(space['n_components'])
        min_cluster_size = random.choice(space['min_cluster_size'])
        random_state = space['random_state']
        
        clusters = generate_clusters(embeddings, 
                                     n_neighbors=n_neighbors, 
                                     n_components=n_components, 
                                     min_cluster_size=min_cluster_size, 
                                     random_state=random_state)
    
        label_count, cost = score_clusters(clusters, prob_threshold = 0.05)
                
        results.append([i, n_neighbors, n_components, min_cluster_size, label_count, cost])
    
    result_df = pd.DataFrame(results, columns=['run_id', 'n_neighbors', 'n_components', 
                                               'min_cluster_size', 'label_count', 'cost'])
    
    return result_df.sort_values(by='cost')

In [ ]:
def objective(params, embeddings, label_lower, label_upper):
    """
    Objective function for hyperopt to minimize

    Arguments:
        params: dict, contains keys for 'n_neighbors', 'n_components',
               'min_cluster_size', 'random_state' and
               their values to use for evaluation
        embeddings: embeddings to use
        label_lower: int, lower end of range of number of expected clusters
        label_upper: int, upper end of range of number of expected clusters

    Returns:
        loss: cost function result incorporating penalties for falling
              outside desired range for number of clusters
        label_count: int, number of unique cluster labels, including noise
        status: string, hypoeropt status

        """
    
    clusters = generate_clusters(embeddings, 
                                 n_neighbors = params['n_neighbors'], 
                                 n_components = params['n_components'], 
                                 min_cluster_size = params['min_cluster_size'],
                                 random_state = params['random_state'])
    
    label_count, cost = score_clusters(clusters, prob_threshold = 0.05)
    
    #15% penalty on the cost function if outside the desired range of groups
    if (label_count < label_lower) | (label_count > label_upper):
        penalty = 0.15 
    else:
        penalty = 0
    
    loss = cost + penalty
    
    return {'loss': loss, 'label_count': label_count, 'status': STATUS_OK}

In [ ]:
def bayesian_search(embeddings, space, label_lower, label_upper, max_evals=100):
    """
    Perform bayesian search on hyperparameter space using hyperopt

    Arguments:
        embeddings: embeddings to use
        space: dict, contains keys for 'n_neighbors', 'n_components',
               'min_cluster_size', and 'random_state' and
               values that use built-in hyperopt functions to define
               search spaces for each
        label_lower: int, lower end of range of number of expected clusters
        label_upper: int, upper end of range of number of expected clusters
        max_evals: int, maximum number of parameter combinations to try

    Saves the following to instance variables:
        best_params: dict, contains keys for 'n_neighbors', 'n_components',
               'min_cluster_size', 'min_samples', and 'random_state' and
               values associated with lowest cost scenario tested
        best_clusters: HDBSCAN object associated with lowest cost scenario
                       tested
        trials: hyperopt trials object for search

        """
    
    trials = Trials()
    fmin_objective = partial(objective, 
                             embeddings=embeddings, 
                             label_lower=label_lower,
                             label_upper=label_upper)
    
    best = fmin(fmin_objective, 
                space = space, 
                algo=tpe.suggest,
                max_evals=max_evals, 
                trials=trials)

    best_params = space_eval(space, best)
    print ('best:')
    print (best_params)
    print (f"label count: {trials.best_trial['result']['label_count']}")
    
    best_clusters = generate_clusters(embeddings, 
                                      n_neighbors = best_params['n_neighbors'], 
                                      n_components = best_params['n_components'], 
                                      min_cluster_size = best_params['min_cluster_size'],
                                      random_state = best_params['random_state'])
    
    return best_params, best_clusters, trials

In [ ]:
def combine_results(df_ground, cluster_dict):
    """
    Returns dataframe of all documents and each model's assigned cluster

    Arguments:
        df_ground: dataframe of original documents with associated ground truth
                   labels
        cluster_dict: dict, keys as column name for specific model and value as
                      best clusters HDBSCAN object

    Returns:
        df_combined: dataframe of all documents with labels from
                     best clusters for each model

    """

    df_combined = df_ground.copy()
    
    for key, value in cluster_dict.items():
        df_combined[key] = value.labels_
    
    return df_combined

In [ ]:
def summarize_results(results_dict, results_df):
    """
    Returns a table summarizing each model's performance compared to ground
    truth labels and the model's hyperparametes

    Arguments:
        results_dict: dict, key is the model name and value is a list of: 
                      model column name in combine_results output, best_params and best_clusters 
                      for each model (e.g. ['label_use', best_params_use, trials_use])
        results_df: dataframe output of combine_results function; dataframe of all documents 
                    with labels from best clusters for each model

    Returns:
        df_final: dataframe with each row including a model name, calculated ARI and NMI,
                  loss, label count, and hyperparameters of best model

    """
    
    summary = []

    for key, value in results_dict.items():
        ground_label = results_df['category'].values
        predicted_label = results_df[value[0]].values
        
        ari = np.round(adjusted_rand_score(ground_label, predicted_label), 3)
        nmi = np.round(normalized_mutual_info_score(ground_label, predicted_label), 3)
        loss = value[2].best_trial['result']['loss']
        label_count = value[2].best_trial['result']['label_count']
        n_neighbors = value[1]['n_neighbors']
        n_components = value[1]['n_components']
        min_cluster_size = value[1]['min_cluster_size']
        random_state = value[1]['random_state']
        
        summary.append([key, ari, nmi, loss, label_count, n_neighbors, n_components, 
                        min_cluster_size, random_state])

    df_final = pd.DataFrame(summary, columns=['Model', 'ARI', 'NMI', 'loss', 
                                              'label_count', 'n_neighbors',
                                              'n_components', 'min_cluster_size',
                                              'random_state'])
    
    return df_final.sort_values(by='NMI', ascending=False)

In [ ]:
def plot_clusters(embeddings, clusters, n_neighbors=15, min_dist=0.1):
    """
    Reduce dimensionality of best clusters and plot in 2D

    Arguments:
        embeddings: embeddings to use
        clusteres: HDBSCAN object of clusters
        n_neighbors: float, UMAP hyperparameter n_neighbors
        min_dist: float, UMAP hyperparameter min_dist for effective
                  minimum distance between embedded points

    """
    umap_data = umap.UMAP(n_neighbors=n_neighbors, 
                          n_components=2, 
                          min_dist = min_dist,  
                          #metric='cosine',
                          random_state=42).fit_transform(embeddings)

    point_size = 100.0 / np.sqrt(embeddings.shape[0])
    
    result = pd.DataFrame(umap_data, columns=['x', 'y'])
    result['labels'] = clusters.labels_

    fig, ax = plt.subplots(figsize=(14, 8))
    outliers = result[result.labels == -1]
    clustered = result[result.labels != -1]
    plt.scatter(outliers.x, outliers.y, color = 'lightgrey', s=point_size)
    plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=point_size, cmap='jet')
    plt.colorbar()
    plt.show()

##### Кластеризация

In [ ]:
# clusters_default = generate_clusters(y, 
#                                      n_neighbors = 3, 
#                                      n_components = 7, 
#                                      min_cluster_size = 2,
#                                      random_state=42)

In [ ]:
# labels_def, cost_def = score_clusters(clusters_default)
# print(labels_def)
# print(cost_def)

In [ ]:
# clusters_default.labels_

In [ ]:
# %%time

# space = {
#         "n_neighbors": range(12,16),
#         "n_components": range(3,7),
#         "min_cluster_size": range(2,16),
#         "random_state": 42
#     }

# random_use = random_search(y, space, 100)

In [ ]:
# random_use

In [ ]:
hspace = {
    "n_neighbors": hp.choice('n_neighbors', range(3,16)),
    "n_components": hp.choice('n_components', range(3,16)),
    "min_cluster_size": hp.choice('min_cluster_size', range(10,16)), # 2, 16
    "random_state": 42
}

label_lower = 30
label_upper = 100
max_evals = 100

In [ ]:
best_params, best_clusters, trials = bayesian_search(y, 
                                                    space=hspace, 
                                                    label_lower=label_lower, 
                                                    label_upper=label_upper, 
                                                    max_evals=max_evals)

In [ ]:
# то что в принте выше топ параметры
trials.best_trial

In [ ]:
best_clusters.labels_[0:100]

In [ ]:
best_n_neighbors = best_params['n_neighbors']
best_params

In [ ]:
plot_clusters(y, best_clusters)
# plot_clusters(y, clusters_default)

In [ ]:
df_w_descr['labels'] = best_clusters.labels_
# df_w_descr['labels'] = clusters_default.labels_

In [ ]:
df_w_descr.head()

In [ ]:
df_w_descr.groupby('labels', as_index=False)[['url_host']].count().sort_values(by='url_host', ascending=False)

In [ ]:
df_w_descr.loc[df_w_descr.labels==5]

In [ ]:
df_w_descr.loc[df_w_descr.labels==15]

In [ ]:
# df_w_descr.to_csv('labels.csv', index=False)

### Кластеры 2 уровня

In [ ]:
labels_gr = df_w_descr.loc[df_w_descr.labels!=-1].groupby(['labels'], as_index=False)[['url_host']].count()

x2 = np.empty([labels_gr.shape[0], 768])

for i, lab in enumerate(labels_gr.labels.unique()):
    rows = list(df_w_descr.loc[df_w_descr.labels==lab].index)
    x2[i] = y[rows, :].mean(axis=0)
    
y2 = x2.copy()

In [ ]:
labels_gr.head()

In [ ]:
print(y2.shape)

In [ ]:
labels_gr.to_csv('labels_gr.csv', index=False)
np.save('y2.npy', y2)

In [ ]:
hspace = {
    "n_neighbors": hp.choice('n_neighbors', range(3,16)), # range(3,16)
    "n_components": hp.choice('n_components', range(3,16)),
    "min_cluster_size": hp.choice('min_cluster_size', range(2,16)), # 2, 16
    "random_state": 42
}

label_lower = 30
label_upper = 100
max_evals = 100


best_params, best_clusters, trials = bayesian_search(y2, 
                                                    space=hspace, 
                                                    label_lower=label_lower, 
                                                    label_upper=label_upper, 
                                                    max_evals=max_evals)

In [ ]:
trials.best_trial

In [ ]:
best_clusters.labels_

In [ ]:
best_params

In [ ]:
plot_clusters(y2, best_clusters)

In [ ]:
labels_gr['labels2'] = best_clusters.labels_

In [ ]:
df_w_descr = df_w_descr.merge(labels_gr[['labels', 'labels2']], how='left')

In [ ]:
df_w_descr.groupby('labels2', as_index=False)[['labels']].count().sort_values(by='labels', ascending=False)

In [ ]:
df_w_descr.head()

In [ ]:
df_w_descr.loc[df_w_descr.labels2==5]

In [ ]:
df_w_descr.labels2 = df_w_descr.labels2.fillna(-1).astype(int)

In [ ]:
# df_w_descr.to_csv('labels.csv', index=False)

### Добавление сжатых векторов сайтов

In [ ]:
umap_embeddings = (umap.UMAP(n_neighbors = best_n_neighbors, 
                            n_components = 24, 
                            metric = 'cosine', 
                            random_state = 42)
                        .fit_transform(y))

In [ ]:
df_umap_embeddings = pd.DataFrame(umap_embeddings)
df_umap_embeddings.head()

In [ ]:
umap_embeddings.shape

In [ ]:
umap_embeddings[0]

In [ ]:
np.save('y_umap.npy', umap_embeddings)

In [ ]:
# df_w_descr = pd.concat([df_w_descr, df_umap_embeddings], axis=1)

In [ ]:
df_w_descr.to_csv('labels.csv', index=False)

### Автолейблинг

In [ ]:
# !pip install -U pip setuptools wheel
# !pip install -U spacy
# !python -m spacy download ru_core_news_sm

In [ ]:
# import collections
# from pathlib import Path
# import numpy as np
# import pandas as pd
# import spacy
# from spacy import displacy

In [ ]:
# nlp = spacy.load("ru_core_news_sm")

In [ ]:
# labels = pd.read_csv('labels.csv')

# df = pd.read_excel('sites.xlsx')
# df_w_descr = df.loc[~df.text.isna()]

# replaces = ['К основным результатамОбратная связь о специальных возможностях  ',
#             'поиск картинки видео карты товары переводчик все сервисы ',
#             'Читать ещё', 'Скрыть Меню ', ' Волгоград Google Bing Сообщить об ошибке Настройки Я.ру',
#             'Быстрый ответ:Google ', 'Быстрый ответ:Сми2 - smi2.ru ', 'Быстрый ответ:Sape.ru ',
#             'Меню ', 'Статья в Википедии ',
#             'Некоторые ссылки отсутствуют в результатах поиска в силу требований применимого законодательства. ',
#             'Подробнее ', 'Навигационный ответ ', 'Владелец сайта предпочёл скрыть описание страницы. ']
# for el in replaces:
#   df_w_descr['text'] = df_w_descr['text'].str.replace(el, '')

# labels = labels.merge(df_w_descr[['text', 'url_host']], how='left')
# labels

In [ ]:
# example_doc = nlp(list(labels['text'])[4])

# print(f'{example_doc}\n')

# for token in example_doc:
#     print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_ , token.is_stop)

In [ ]:
# def get_group(df, category_col, category):
#     """
#     Returns documents of a single category
    
#     Arguments:
#         df: pandas dataframe of documents
#         category_col: str, column name corresponding to categories or clusters
#         category: int, cluster number to return
#     Returns:
#         single_category: pandas dataframe with documents from a single category
#     """
    
#     single_category = df[df[category_col]==category].reset_index(drop=True)

#     return single_category 

# def most_common(lst, n_words):
#     """
#     Get most common words in a list of words
    
#     Arguments:
#         lst: list, each element is a word
#         n_words: number of top common words to return
    
#     Returns:
#         counter.most_common(n_words): counter object of n most common words
#     """
#     counter=collections.Counter(lst)
#     return counter.most_common(n_words)

# def extract_labels(category_docs, print_word_counts=False):
#     """
#     Extract labels from documents in the same cluster by concatenating
#     most common verbs, ojects, and nouns

#     Argument:
#         category_docs: list of documents, all from the same category or
#                        clustering
#         print_word_counts: bool, True will print word counts of each type in this category

#     Returns:
#         label: str, group label derived from concatentating most common
#                verb, object, and two most common nouns

#     """

#     verbs = []
#     dobjs = []
#     nouns = []
#     adjs = []
    
#     verb = ''
#     dobj = ''
#     noun1 = ''
#     noun2 = ''

#     # for each document, append verbs, dobs, nouns, and adjectives to 
#     # running lists for whole cluster
#     try:
#       for i in range(len(category_docs)):
#           doc = nlp(category_docs[i])
#           for token in doc:
#               if token.is_stop==False:
#                   if token.dep_ == 'ROOT':
#                       verbs.append(token.text.lower())

#                   elif token.dep_=='dobj':
#                       dobjs.append(token.lemma_.lower())

#                   elif token.pos_=='NOUN':
#                       nouns.append(token.lemma_.lower())
                      
#                   elif token.pos_=='ADJ':
#                       adjs.append(token.lemma_.lower())
#     except:
#       print("Something went wrong in doc", category_docs[i])

#     # for printing out for inspection purposes
#     if print_word_counts:
#         for word_lst in [verbs, dobjs, nouns, adjs]:
#             counter=collections.Counter(word_lst)
#             print(counter)
    
#     # take most common words of each form
#     if len(verbs) > 0:
#         verb = most_common(verbs, 1)[0][0]
    
#     if len(dobjs) > 0:
#         dobj = most_common(dobjs, 1)[0][0]
    
#     if len(nouns) > 0:
#         noun1 = most_common(nouns, 1)[0][0]
    
#     if len(set(nouns)) > 1:
#         noun2 = most_common(nouns, 2)[1][0]
    
#     # concatenate the most common verb-dobj-noun1-noun2 (if they exist)
#     label_words = [verb, dobj]
    
#     for word in [noun1, noun2]:
#         if word not in label_words:
#             label_words.append(word)
    
#     if '' in label_words:
#         label_words.remove('')
    
#     label = '_'.join(label_words)
    
#     return label

# def apply_and_summarize_labels(df, category_col):
#     """
#     Assign groups to original documents and provide group counts

#     Arguments:
#         df: pandas dataframe of original documents of interest to
#             cluster
#         category_col: str, column name corresponding to categories or clusters

#     Returns:
#         summary_df: pandas dataframe with model cluster assignment, number
#                     of documents in each cluster and derived labels
#     """
    
#     numerical_labels = df[category_col].unique()
    
#     # create dictionary of the numerical category to the generated label
#     label_dict = {}
#     for label in numerical_labels:
#         current_category = list(get_group(df, category_col, label)['text'])
#         label_dict[label] = extract_labels(current_category)
        
#     # create summary dataframe of numerical labels and counts
#     summary_df = (df.groupby(category_col)['text'].count()
#                     .reset_index()
#                     .rename(columns={'text':'count'})
#                     .sort_values('count', ascending=False))
    
#     # apply generated labels
#     summary_df['label'] = summary_df.apply(lambda x: label_dict[x[category_col]], axis = 1)
    
#     return summary_df

# def combine_ground_truth(df_clusters, df_ground, key):
#     """
#     Combines dataframes of documents with extracted and ground truth labels
    
#     Arguments:
#         df_clusters: pandas dataframe, each row as a document with corresponding extracted label
#         df_ground: pandas dataframe, each row as a document with corresponding ground truth label
#         key: str, key to merge tables on
        
#     Returns:
#         df_combined: pandas dataframe, each row as a document with extracted and ground truth labels
#     """
#     df_combined = pd.merge(df_clusters, df_ground, on=key, how = 'left')
#     return df_combined

# def get_top_category(df_label, df_summary):
#     """
#     Returns a dataframe comparing a single model's results to ground truth
#     label to evalute cluster compositions and derived label relative to labels
#     and counts of most commmon ground truth category

#     Arguments:
#         df_label: pandas dataframe, each row as a document with extracted and ground truth labels
#                   (result of `combine_ground_truth` function)
#         df_summary: pandas dataframe with model cluster assignment, number
#                     of documents in each cluster and derived labels
#                     (result from `apply_and_summarize_labels` function)

#     Returns:
#         df_result: pandas dataframe with each row containing information on
#                    each cluster identified by this model, including count,
#                    extracted label, most represented ground truth label name,
#                    count and percentage of that group
#     """
#     df_label_ground = (df_label.groupby('label')
#                       .agg(top_ground_category=('category', lambda x:x.value_counts().index[0]), 
#                            top_cat_count = ('category', lambda x:x.value_counts()[0]))
#                       .reset_index())
    
#     df_result = pd.merge(df_summary, df_label_ground, on='label', how='left')
#     df_result['perc_top_cat'] = df_result.apply(lambda x: int(round(100*x['top_cat_count']/x['count'])), axis=1)
    
#     return df_result



In [ ]:
# example_category = list(get_group(labels, 'labels', 10)['text'])
# extract_labels(example_category, True)

In [ ]:
# cluster_summary = apply_and_summarize_labels(labels, 'labels')
# cluster_summary

In [ ]:
# labels = labels.merge(cluster_summary[['labels', 'label']], how='left')

In [ ]:
# labels[['url_host', 'labels', 'label']].to_csv('labels.csv', index=False)